# Preparación de Proteínas para Docking y DM
Se realiza la preparación final de las proteínas generadas con Modeller, añadiendo átomos faltantes e hidrógenos a las estructuras.

In [1]:
import pdb2pqr
import pdb4amber
import os
from pathlib import Path
import glob

/home/joel/anaconda3/envs/mds/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [8]:
# Nombre de la proteina de interes
prot_name = 'erk2'
# Secuencia P28482 (ERK2_HUMAN)
uniprot_id = "P28482"

## Se generan las estructuras preparadas 
### (pdb4amber, reduce, pdb2pqr y propka)
La siguiente función recibe como entrada cada una de las estructuras generadas con Modeller (a alas cuales no se le agregaron hidrógenos) y devuelve:
- **Archivos ID_ENS.pdb**: La estructura de la proteína con **todos los hidrógenos añadidos**, pero **no optimizados** para los residuos ionizables. Esto implica que todos sus residuos de histidina serán **HIS**. Además, **todas las conformaciones de la proteína tendrán el mismo número de átomos**, y en las mismas posiciones, lo que permitirá crear un ensamble con ellas.
- **Archivos ID_PROT.pdb**: Estructura de la proteína con hidrógenos añadidos a un pH = 7, para lo cual se ha usado la herramienta **_reduce_** incluida en AmberTools y **PDB2PQR**. Pueden ser usados como estructura inicial para _docking_ y dinámica molecular.
- **Archivos ID_PQR.pdb y ID_PQR.propka**: Archivos resultantes tras la ejecuación de **PDB2PQR** y **propka**.

### Creación de directorios
Lista de modelos generados por Modeller.

In [39]:
# Lista de archivos pdb para preparar, originalmente modelos generados con Modeller
pdb_list = glob.glob(F"../ARCHIVOS/CRISTALES/PROT_{prot_name.upper()}_MODELOS_modeller_NoPrep/*")
pdb_list.sort()

Se crean las rutas a los directorios, y los directorios, si aún no han sido creados.

In [27]:
main_out_dir = F"../ARCHIVOS/CRISTALES/PROT_{prot_name.upper()}_PREPARADAS/"
out_dir_ens = main_out_dir +  "PREP_ALL_ENSAMBLE/" # Hidrógenos agregados sin pka
out_dir_PREP_PROT = main_out_dir + "PREP_PH_7/" # Hidrógenos agregados usando Propka (res ionizables a pH 7)
out_dir_pqr = main_out_dir + "PQR_PKAs/" # Directorio de pqr"
# Creación de los directorios, si estos no exiten
import os
if not os.path.exists(main_out_dir):
    os.makedirs(main_out_dir)
    os.makedirs(out_dir_ens)
    os.makedirs(out_dir_PREP_PROT)
    os.makedirs(out_dir_pqr)

Procedemos a realizar la preparación de cada uno delos modelos de la proteína.

In [60]:
# Sufijos para nombrar los archivos resultantes
ens_f_tail = "_ENS"
prep_f_tail = "_PROT"
prep_anyways = False # True para volver a preparar TODAS las proteínas

num_struct = [16, 34, 35, 36, 38, 39, 54, 61, 64, 86, 90, 91, 92, 154]
for pdb_file in np.array(pdb_list)[num_struct]:
    pdb_id = Path(pdb_file).stem.rsplit('_', 1)[0]
    print(F"Preparando la proteína: {pdb_id}.")
    # Omite el proceso si la proteína ya ha sido preparada previamente
    if not prep_anyways:
        if os.path.isfile(out_dir_PREP_PROT  + pdb_id + prep_f_tail + ".pdb") \
            and os.path.isfile(out_dir_ens + pdb_id + ens_f_tail + ".pdb"):
            print(F"La proteína {pdb_id} ya existe en los directorios\n")
            continue
    
    # ENSAMBLE: Añade átomos faltantes, incluyendo Hs
    # No toma en cuenta los estados ionizables
    ensamble_file = out_dir_ens + pdb_id + ens_f_tail + ".pdb"
    pdb4amber.run(arg_pdbin = pdb_file,
                  arg_add_missing_atoms = True, 
                  arg_pdbout = ensamble_file)
    # PROT PREPARADA:
    # Ejecuta pdb2pqr para obtener los estados de protonación de los residuos ionizables
    pqr_file = out_dir_pqr + pdb_id + "_PQR.pdb"
    os.system("pdb2pqr_cli --ff=amber --with-ph=7.0 --ffout=amber --ph-calc-method=propka --chain "
               + pdb_file + " " 
               + pqr_file )
    # Vuelve a ejecutar pdb4amber para generar las PROT con los estados ionizables usando "--reduce"
    prepared_file = out_dir_PREP_PROT  + pdb_id + prep_f_tail + ".pdb"
    pdb4amber.run(arg_pdbin  = pdb_file,
                  arg_reduce = True,
                  arg_pdbout = prepared_file )
    # Elimina los archivos temporales de pdb4amber
    os.system("rm " + out_dir_ens + "/*" + ens_f_tail + "_*")
    os.system("rm " + out_dir_PREP_PROT + "/*" + prep_f_tail + "_*")

Preparando la proteína: 3i60.
Preparando la proteína: 4fv4.
Preparando la proteína: 4fv5.
Preparando la proteína: 4fv6.
Preparando la proteína: 4fv8.
Preparando la proteína: 4fv9.
Preparando la proteína: 4o6e.
Preparando la proteína: 4qp8.
Preparando la proteína: 4qta.
Preparando la proteína: 5ax3.
Preparando la proteína: 5bvd.
Preparando la proteína: 5bve.
Preparando la proteína: 5bvf.
Preparando la proteína: 6ots.
